In [1]:
# import libraries 
import pandas as pd
import numpy as np
import math
from numpy.linalg import norm

pd.set_option('display.max_columns', None)

np.random_state = 42

## Load in `sd_pp` Dataset

In [2]:
# load in df with the cluster labels
sd_pp = pd.read_csv('data/sd_pp', index_col = 0)
sd_pp

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,-1.152181,-0.561263,-0.534033,-0.728248,2.546108,0.605540,1.504754,-0.909516,-0.562984,-0.490416,3.516369,-0.408425,0.513223,-0.764034,-0.238620,0.038201,-0.972836,-0.581030,-0.581030,0.372804,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,0.970300,0.0,-0.16778,-0.183109
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### Get the `listing_url`

In [3]:
# load in df with the listing urls
sd_listings_url = pd.read_csv('data/sd_modeling_with_urls', index_col= 0)
sd_listings_url = sd_listings_url[['listing_url']]
sd_listings_url = sd_listings_url.reset_index(drop = 'index')
sd_listings_url

,listing_url
0,https://www.airbnb.com/rooms/33159143
1,https://www.airbnb.com/rooms/17138468
2,https://www.airbnb.com/rooms/21898446
3,https://www.airbnb.com/rooms/25948680
4,https://www.airbnb.com/rooms/1756516
...,...
13034,https://www.airbnb.com/rooms/20473217
13035,https://www.airbnb.com/rooms/17975465
13036,https://www.airbnb.com/rooms/4227110
13037,https://www.airbnb.com/rooms/17975942


## Create a new dataframe for comparison listing

**Select a listing**

In [4]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
12390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.838521,3.305887,2.813687,1.376162,0.680634,-0.078531,1.504754,4.812091,3.311218,-0.490416,-0.238541,0.389161,0.486436,-0.764034,2.053416,0.038201,-0.972836,-0.447771,-0.447771,-0.092556,0.368466,-2.234044,0.272019,0.278668,0.342562,-0.717556,0.0,0.9703,0.0,-0.16778,-0.183109


### Convert to Arrays

In [5]:
# convert single listing to an array
random_listing_array = random_listing.values

# convert all listings to an array
sd_pp_array = sd_pp.values

In [6]:
print(random_listing_array.shape)
print(sd_pp_array.shape) 

(1, 240)
(13039, 240)


## Determine Cosine Similarity

In [7]:
# define two lists or arrays to compare
A = np.squeeze(np.asarray(sd_pp_array))
B = np.squeeze(np.asarray(random_listing_array))
print("A:\n", A)
print("B:\n", B)
 s
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))
print("Cosine Similarity:\n", cosine)

A:
 [[ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 ...
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]
 [ 0.          0.          0.         ...  0.         -0.16778002
  -0.18310885]]
B:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          0.          0.     

### Add Cosine Similarity to `sd_clustered`

In [8]:
# load in df with the cluster labels
sd_clustered = pd.read_csv('data/sd_clustered', index_col = 0)
sd_clustered.reset_index(drop = 'index')

,id,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,33159143,0,32.91736,-117.07635,Scripps Ranch,92131,House,Private room,1,1.0,1.0,1,Real Bed,1100.0,1100.0,1000.0,0.0,1.0,0.0,60.0,90.0,within a few hours,100.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
1,17138468,0,32.84067,-117.27443,La Jolla,92037,Apartment,Entire home/apt,1,2.0,2.0,3,Real Bed,1400.0,1400.0,0.0,0.0,1.0,0.0,30.0,1125.0,no response,0.0,0.0,1.0,1.0,0.0,2.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
2,21898446,3,32.79797,-117.24250,Pacific Beach,92109,Townhouse,Private room,1,1.0,1.0,1,Real Bed,1250.0,1250.0,0.0,0.0,1.0,0.0,30.0,30.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
3,25948680,3,32.77545,-117.05923,College Area,92120,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,1150.0,1150.0,0.0,0.0,1.0,0.0,31.0,40.0,no response,0.0,0.0,1.0,1.0,0.0,0.0,0.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,0.0,0.0,flexible,0.0,0.0
4,1756516,2,32.84619,-117.27558,La Jolla,92037,Condominium,Private room,1,1.0,1.0,1,Real Bed,110.0,110.0,200.0,0.0,1.0,50.0,180.0,365.0,within a day,67.0,0.0,6.0,1.0,1.0,2.0,4.0,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0.0,0.0,0.0,strict_14_with_grace_period,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13034,20473217,1,32.78399,-117.25391,Mission Bay,92109,Villa,Entire home/apt,20,7.0,8.0,11,Real Bed,3000.0,3000.0,3000.0,0.0,20.0,0.0,1.0,1125.0,within an hour,95.0,1.0,86.0,1.0,1.0,0.0,0.0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0.0,0.0,0.0,luxury_moderate,0.0,0.0
13035,17975465,1,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,6.0,12.0,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
13036,4227110,1,32.79395,-117.23492,Mission Bay,92109,House,Entire home/apt,20,2.0,4.0,10,Real Bed,208.0,208.0,1000.0,332.0,4.0,25.0,1.0,1125.0,within an hour,96.0,0.0,11.0,1.0,1.0,142.0,284.0,92.0,9.0,9.0,9.0,9.0,10.0,8.0,0.0,1.0,0.0,strict_14_with_grace_period,0.0,0.0
13037,17975942,1,32.83598,-117.27878,La Jolla,92037,House,Entire home/apt,21,5.5,7.0,13,Real Bed,1305.0,1305.0,1000.0,800.0,21.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,4.0,8.0,80.0,9.0,8.0,10.0,8.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0


#### Adding in `listing_url`

In [10]:
# merge on index
sd_clustered = sd_clustered.join(sd_listings_url)

In [11]:
# add similarity 
rec = sd_clustered.copy()
rec['similarity'] = pd.DataFrame(cosine).values

In [12]:
# reorder column names
rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]

## Get Recommendations!

In [13]:
# sort by highest similarity
rec.sort_values(by = ['similarity'], ascending = False).head(6)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
12400,17975433,https://www.airbnb.com/rooms/19699773,1.000000,1,32.84374,-117.26622,La Jolla,92037,House,Entire home/apt,10,4.5,5.0,5,Real Bed,458.0,458.0,1000.0,599.0,10.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,8.0,16.0,95.0,10.0,8.0,10.0,10.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
12411,19699779,https://www.airbnb.com/rooms/29556970,0.962233,1,32.81931,-117.26941,La Jolla,92037,House,Entire home/apt,10,3.0,3.0,5,Real Bed,405.0,405.0,1000.0,499.0,10.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,8.0,16.0,93.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
12656,17975442,https://www.airbnb.com/rooms/10729725,0.955187,1,32.82903,-117.25679,La Jolla,92037,House,Entire home/apt,12,4.5,5.0,6,Real Bed,994.0,994.0,1000.0,749.0,12.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,3.0,6.0,93.0,10.0,9.0,10.0,10.0,9.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
12708,34469698,https://www.airbnb.com/rooms/34250369,0.954657,1,32.71578,-117.24861,Wooded Area,92106,House,Entire home/apt,12,3.5,5.0,6,Real Bed,507.0,507.0,1000.0,699.0,12.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,1.0,2.0,100.0,10.0,8.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
12384,17975460,https://www.airbnb.com/rooms/37457808,0.941368,1,32.82893,-117.27708,La Jolla,92037,House,Entire home/apt,10,2.5,4.0,5,Real Bed,538.0,538.0,1000.0,449.0,10.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,4.0,8.0,95.0,9.0,9.0,10.0,10.0,10.0,9.0,0.0,1.0,0.0,super_strict_60,0.0,0.0
11558,31416675,https://www.airbnb.com/rooms/32497274,0.938751,1,32.83143,-117.27849,La Jolla,92037,House,Entire home/apt,8,4.5,4.0,5,Real Bed,606.0,606.0,1000.0,534.0,8.0,0.0,1.0,1125.0,within an hour,99.0,0.0,403.0,1.0,0.0,0.0,0.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,super_strict_60,0.0,0.0


### Get Recommendations Function

In [43]:
# selects a random listing from sd_clustered
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
2745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.819881,-0.561263,-0.534033,-0.728248,-0.446787,-0.417369,-0.555421,-0.909516,-0.562984,-0.490416,-0.174899,-0.454662,0.513223,1.308842,-0.23862,0.038201,-0.972836,0.884821,0.884821,0.527923,0.368466,0.464525,0.272019,0.278668,0.342562,0.598182,0.0,0.9703,0.0,-0.16778,-0.183109


In [44]:
def get_recommendations(df, listing):
    """
    Takes in preprocessed dataframe and selected listing as inputs and gives top 5 (including listing)
    recommendations based on cosine similarity. 
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # convert single listing to an array
    listing_array = listing.values

    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_clustered.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # reorder column names
    rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(6)


In [45]:
get_recommendations(sd_pp, random_listing)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
2745,14965205,https://www.airbnb.com/rooms/17830608,1.000000,3,32.71664,-117.12864,South Park,92102,Guest suite,Private room,2,1.0,1.0,1,Real Bed,70.0,140.0,0.0,0.0,1.0,0.0,2.0,30.0,within an hour,100.0,1.0,1.0,1.0,0.0,88.0,176.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
2619,23361007,https://www.airbnb.com/rooms/24245439,0.929596,3,32.71933,-117.12296,South Park,92102,House,Private room,2,1.0,1.0,1,Real Bed,75.0,150.0,0.0,35.0,1.0,0.0,2.0,7.0,within an hour,100.0,1.0,1.0,1.0,0.0,78.0,156.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
4173,22131133,https://www.airbnb.com/rooms/25323073,0.883493,3,32.71682,-117.12863,South Park,92102,Condominium,Private room,2,1.0,1.0,2,Real Bed,55.0,55.0,0.0,20.0,1.0,15.0,1.0,6.0,within an hour,100.0,1.0,1.0,1.0,0.0,93.0,186.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,moderate,0.0,0.0
5229,14057546,https://www.airbnb.com/rooms/36636389,0.856223,3,32.72233,-117.12342,South Park,92102,House,Entire home/apt,3,1.0,1.0,1,Real Bed,95.0,190.0,95.0,40.0,1.0,0.0,2.0,14.0,within an hour,90.0,1.0,1.0,1.0,0.0,92.0,184.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
2881,26897579,https://www.airbnb.com/rooms/29081352,0.840597,3,32.75266,-117.12674,North Hills,92104,Guest suite,Private room,2,1.0,1.0,1,Real Bed,65.0,130.0,0.0,30.0,1.0,0.0,2.0,1125.0,within an hour,100.0,1.0,2.0,1.0,0.0,74.0,148.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
3866,15159471,https://www.airbnb.com/rooms/19735111,0.812623,3,32.76463,-117.13147,North Hills,92116,House,Private room,2,1.0,1.0,1,Real Bed,70.0,70.0,0.0,0.0,1.0,0.0,1.0,4.0,within an hour,100.0,1.0,1.0,1.0,0.0,104.0,208.0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,1.0,0.0,flexible,0.0,0.0
